In [6]:
import requests
from bs4 import BeautifulSoup
import re, nltk
import io

In [20]:
def get_urls(url):
    '''get a list of debate urls from the debate page
    params: url - debate url
    returns: a list of url strings
    '''
    r = requests.get(url)
    data = r.text
    soup = BeautifulSoup(data, 'lxml')
    urls = []
    for link in soup.find_all('a'):
        link_string = link.get('href')
        if '/ws/' in link_string:
            urls.append(link_string)
    return urls

def get_debate(url):
    try:
        r = requests.get(url).text
        soup = BeautifulSoup(r, 'lxml')
        letters = soup.find_all("span", class_="displaytext")
        text = letters[0].get_text().strip()
        turns = re.split("([A-Z]+:)",text)
        string_blob = ''
        for turn in turns:
            if len(turn) > 0:
                string_blob += turn.strip() +'\n'
                
        date = soup.find_all("meta")
        meta_data = str(date[-1])
        title = get_file_title(meta_data)
        return string_blob, title
    except:
        return None, None

def get_file_title(meta_data):
    party = ""
    if 'Republican' in meta_data:
        party ='R'
    elif 'Democrat' in meta_data:
        party = 'D'
    else:
        party = None
        
    if 'undercard' in meta_data:
        # undercard debates for overflow
        group = 'u'
    else:
        # main
        group = 'm'
        
    try:
        cur_date = re.search('[A-Z][a-z]+ [0-9]+, [0-9]+',meta_data).group(0)
        cur_date = re.sub(',','',cur_date)
        cur_date = re.sub(' +','_',cur_date)
    except:
        cur_date = None
        
    if party is not None and cur_date is not None and group is not None:
        new_title = party + '_' + cur_date + '_' + group
        return new_title
    else:
        return None

In [21]:
urls = get_urls('http://www.presidency.ucsb.edu/debates.php')
for url in urls:
    debate, title = get_debate(url)
    if debate is not None and title is not None:
        filename = "debates/{0}.txt".format(title)
        with io.open(filename,'w',encoding='utf8') as f:
            f.write(debate)

In [40]:
import io
with io.open(filename,'r',encoding='utf8') as f:
    text = f.read()
# process Unicode text
with io.open(filename,'w',encoding='utf8') as f:
    f.write(text)